In [243]:
import mido
import numpy as np

MAX_NUM_OF_CHANNELS = 16
DRUM_CHANNEL = 9

# return true if the message is a note
def is_note(msg):
    return msg.type == 'note_on' or msg.type == 'note_off'

# return true if the message indicates the start of a note
# return false if the message indicates the end of a note
def is_note_on(msg):
    return msg.type == 'note_on' and msg.velocity != 0

def is_drum(msg):
    return msg.channel == DRUM_CHANNEL

# get the time and tick indicating the bars, and their beat and tempo
def get_bars_info(midi_file):
    
    ticks_per_beat = midi_file.ticks_per_beat

    total_tick = 0
    total_time = 0
    rhythm_events = []

    # store the time signature and tempo change events as rhythm_events
    for msg in midi_file:

        # sum up the tick/time
        if msg.time != 0:
            total_tick += mido.second2tick(msg.time, ticks_per_beat, current_tempo)
            total_time += msg.time

        # store meta message of time signature and tempo with current tick
        if msg.is_meta:

            if msg.type == 'time_signature':
                time_signature_quarter = msg.numerator / (msg.denominator / 4)
                rhythm_events.append([total_tick, -1, time_signature_quarter])

            if msg.type == 'set_tempo':
                current_tempo = msg.tempo
                rhythm_events.append([total_tick, current_tempo, -1])
    
    # store the tick of the end of midi
    rhythm_events.append([total_tick, -1, -1])

    cumulative_time = 0
    cumulative_tick = 0
    prev_tick = 0
    current_tempo = -1
    current_ts_quarter = -1
    bars_time = []
    bars_tick = []
    bars_beat = []
    bars_tempo = []

    # append the time/tick at each bar according to 'rhythm_events'
    for rhythm_event in rhythm_events:

        # detect rhythm change, then conclude the bars before the change
        if rhythm_event[0] != prev_tick:

            tick_diff = rhythm_event[0] - prev_tick
            prev_tick = rhythm_event[0]

            # conclude the bar time/tick by dividing the bar length
            # given total tick, and constant beat and tempo, the bars can be separated by dividing evenly
            num_of_bar = tick_diff / ticks_per_beat
            num_of_bar_2 = num_of_bar
            while num_of_bar > 0:
                bar_tick = (num_of_bar_2 - num_of_bar) * ticks_per_beat
                bars_time.append(mido.tick2second(bar_tick, ticks_per_beat, current_tempo) + cumulative_time)
                bars_tick.append(bar_tick + cumulative_tick)
                bars_beat.append(current_ts_quarter)
                bars_tempo.append(current_tempo)
                num_of_bar -= current_ts_quarter            

            cumulative_time += mido.tick2second(tick_diff, ticks_per_beat, current_tempo)
            cumulative_tick += tick_diff

        # update the time signature and time tempo events
        if rhythm_event[1] != -1:
            current_tempo = rhythm_event[1]
        if rhythm_event[2] != -1:
            current_ts_quarter = rhythm_event[2]

    return bars_time, bars_tick, bars_beat, bars_tempo

# get the details of notes in each bar
# i.e. bars_note_info[bar index] = [start tick of the bar, end tick of the bar, bar_note_info]
# i.e. bar_note_info[note index] = [channel index, note no., triggered time (in tick), 1 = 'note_on' / 0 = 'note_off']
def get_bars_note_info(midi_file):

    _, bars_tick, bars_beat, _ = get_bars_info(midi_file)

    ticks_per_beat = midi_file.ticks_per_beat
    current_tempo = -1
    current_ts_quarter = -1

    total_tick = 0
    bar_start_tick = bars_tick.pop(0)
    bar_note_info = []
    bars_note_info = []

    # store the time signature and tempo change events as rhythm_events
    for msg in midi_file:

        # sum up the tick/time
        if msg.time != 0:        
            total_tick += mido.second2tick(msg.time, ticks_per_beat, current_tempo)
            if len(bars_tick) != 0 and total_tick >= bars_tick[0]:

                # find the suitable bar for placing current note
                bars_shift_idx = 0
                while len(bars_tick) > (bars_shift_idx + 1) and total_tick >= bars_tick[bars_shift_idx + 1]:
                    bars_shift_idx += 1

                bars_note_info.append([bar_start_tick, bars_tick[0] - bar_start_tick, bar_note_info, bars_beat[0]])
                bar_note_info = []
                bar_start_tick = bars_tick.pop(0)
                bars_beat.pop(0)

                # put back the shifted (empty) bars
                for pad_bar_idx in range(bars_shift_idx):
                    bars_note_info.append([bar_start_tick, bars_tick[0] - bar_start_tick, bar_note_info, bars_beat[0]])
                    bar_note_info = []
                    bar_start_tick = bars_tick.pop(0)
                    bars_beat.pop(0)


        # store meta message of time signature and tempo with current tick
        if msg.is_meta:

            if msg.type == 'time_signature':
                current_ts_quarter = msg.numerator / (msg.denominator / 4)

            if msg.type == 'set_tempo':
                current_tempo = msg.tempo

        if not msg.is_meta:

            if is_note(msg):

                if is_note_on(msg):
                    bar_note_info.append([msg.channel, msg.note, total_tick, 1, msg.velocity])

                else:
                    bar_note_info.append([msg.channel, msg.note, total_tick, 0, msg.velocity])
    return bars_note_info

# get rhythm pattern in each bar of each channel
# i.e. tracks_rhythm_pattern[channel index, bar index] = [[note1 start_time, note1 duration], [note2 start_time, note2 duration], ...]
def get_rhythm_pattern(bars_note_info):

    MAX_NUM_OF_CHANNELS = 16
    MAX_NUM_OF_NOTE_IN_BAR = 64
    tracks_rhythm_pattern = np.zeros([MAX_NUM_OF_CHANNELS, len(bars_note_info), MAX_NUM_OF_NOTE_IN_BAR, 5]) - 1
    # print(tracks_rhythm_pattern.shape)


    prev_bar_notes = []

    for idx, bar_note_info in enumerate(bars_note_info):

        if len(prev_bar_notes) != 0:
            for prev_bar_note in prev_bar_notes:
                append_idx = 0
                while append_idx < 64:
                    if tracks_rhythm_pattern[prev_bar_note[0], idx, append_idx, 0] == -1:
                        tracks_rhythm_pattern[prev_bar_note[0], idx, append_idx] = [0, prev_bar_note[1], 0, 0, 0]
                        break
                    append_idx += 1
            prev_bar_notes = []

        # print(idx, bar_note_info[2])
        for note in bar_note_info[2]:
            if note[3]:
                append_idx = 0
                while append_idx < 64:
                    if tracks_rhythm_pattern[note[0], idx, append_idx, 0] == -1:
                        tracks_rhythm_pattern[note[0], idx, append_idx, 0] = note[2] - bar_note_info[0]
                        tracks_rhythm_pattern[note[0], idx, append_idx, 1] = -2 - note[1]
                        break
                    append_idx += 1
            else:
                # print(tracks_rhythm_pattern[note[0], idx, :, 1], -2 - note[1])
                relative_idx = np.nonzero(tracks_rhythm_pattern[note[0], idx, :, 1] == -2 - note[1])[0][0]
                tracks_rhythm_pattern[note[0], idx, relative_idx, 1] = note[2] - bar_note_info[0] - tracks_rhythm_pattern[note[0], idx, relative_idx, 0]
                # print(tracks_rhythm_pattern[note[0], idx, relative_idx])

        for channel_idx in range(MAX_NUM_OF_CHANNELS):

            temp_idx = 0

            while temp_idx < MAX_NUM_OF_NOTE_IN_BAR and tracks_rhythm_pattern[channel_idx, idx, temp_idx, 1] != -1:
                if tracks_rhythm_pattern[channel_idx, idx, temp_idx, 1] < -1:
                    prev_bar_notes.append([channel_idx, tracks_rhythm_pattern[channel_idx, idx, temp_idx, 1]])
                    tracks_rhythm_pattern[channel_idx, idx, temp_idx, 1] = bar_note_info[1] - tracks_rhythm_pattern[note[0], idx, relative_idx, 0]
                    # print(tracks_rhythm_pattern[channel_idx, idx, temp_idx])
                tracks_rhythm_pattern[channel_idx, idx, temp_idx] /= bar_note_info[1]
                temp_idx += 1
        # print(prev_bar_notes)
    
    return tracks_rhythm_pattern

def get_midi_info(bars_note_info):
    midi_info = np.empty((len(bars_note_info), MAX_NUM_OF_CHANNELS), dtype=np.ndarray)
    for i in range(midi_info.shape[0]):
        for j in range(midi_info.shape[1]):
            midi_info[i, j] = np.empty((0, 7))


    prev_bar_notes = []

    for idx, bar_note_info in enumerate(bars_note_info):

        if len(prev_bar_notes) != 0:
            for prev_bar_note in prev_bar_notes:
                midi_info[idx, prev_bar_note[0]] = np.vstack((midi_info[idx, prev_bar_note[0]], [prev_bar_note[2], prev_bar_note[3], 0, prev_bar_note[1], 0, 0, 1]))
            prev_bar_notes = []

        for note in bar_note_info[2]:
            if note[3]:
                midi_info[idx, note[0]] = np.vstack((midi_info[idx, note[0]], [note[1], note[4], note[2] - bar_note_info[0], -2 - note[1], 0, 0, 0]))
            else:
                relative_idx = np.nonzero(midi_info[idx, note[0]][:, 3] == -2 - note[1])[0][0]
                midi_info[idx, note[0]][relative_idx, 3] = note[2] - bar_note_info[0] - midi_info[idx, note[0]][relative_idx, 2]

        for channel_idx in range(MAX_NUM_OF_CHANNELS):

            for i, info in enumerate(midi_info[idx, channel_idx]):
                if info[3] < -1:
                    prev_bar_notes.append([channel_idx, info[3], info[0], info[1]])
                    midi_info[idx, channel_idx][i, 3] = bar_note_info[1] - info[2]
                    midi_info[idx, channel_idx][i, 6] = 1
            midi_info[idx, channel_idx][:, 2:4] /= bar_note_info[1]
            midi_info[idx, channel_idx][:, 4:6] = midi_info[idx, channel_idx][:, 2:4] * bar_note_info[3]
    return midi_info

# turn numeric notes into english notes, octave info not preserved
def note_int2char(note):
    return {
        '0': 'C', '1': 'Db', '2': 'D', '3': 'Eb',
        '4': 'E', '5': 'F', '6': 'F#', '7': 'G',
        '8': 'Ab', '9': 'A', '10': 'Bb', '11': 'B'
    }.get(str(note % 12), 'undef')

# define the chords and its relative position
chords = np.empty((0, 12), dtype=np.int8)

chords_entries = np.array([
    # C1
    [6, -1, 0, -4, -4, 0, -1, -5, -1, -1, -3, -3],
    # C5
    [5, -1, 0, -4, -4, 0, -1, 5, -1, -1, -3, -3],
    # C
    [5, -3, 0, -4, 4, 0, -3, 4, -3, 1, -4, -4],
    # Cm
    [5, -3, 0, 4, -4, 0, -3, 4, 1, 0, -4, -4],
    # C7
    [5, -3, 0, -3, 3, 0, -3, 3, 1, 1, 3, -4],
    # CMaj7
    [5, -3, 0, -3, 3, 0, -3, 3, -3, 1, -3, 3],
    # Cmin7
    [5, -3, 0, 3, -3, 0, -3, 3, 1, 0, 3, -1],
    # Cdim
    [5, 0, -3, 4, -3, -3, 4, -4, 0, -3, -3, -3],
], dtype=np.int8)

for chords_entry in chords_entries:
    for i in range(12):
        chords = np.vstack((chords, np.roll(chords_entry, i)))

# name the chord
def name_chord(chord):
    return {
        '0': note_int2char(chord) + '1',
        '1': note_int2char(chord) + '5',
        '2': note_int2char(chord),
        '3': note_int2char(chord) + 'm',
        '4': note_int2char(chord) + '7',
        '5': note_int2char(chord) + 'M7',
        '6': note_int2char(chord) + 'm7',
        '7': note_int2char(chord) + 'dim'
    }.get(str(chord // 12), 'undef')

# chords to key, '[C, Db, D, ..., B, Cm, Dbm, ..., Bbm, Bm]'
chord_to_key = np.empty((0, 24), dtype=np.int8)

chord_to_key_entries = np.array([
    # C note only
    [2, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 2, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0],
    # C power chords
    [2, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 2, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0],
    # C Major chords
    [2, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0],
    # C Minor chords
    [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
    # C Dominant 7th chords
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    # C Major 7th chords
    [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
    # C Minor 7th chords
    [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
    # C Diminished chords
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
], dtype=np.int8)

for chord_to_key_entry in chord_to_key_entries:
    for i in range(12):
        chord_to_key = np.vstack((chord_to_key, np.concatenate((np.roll(chord_to_key_entry[0:12], i), np.roll(chord_to_key_entry[12:24], i)))))

# name the key
def name_key(key):
    return {
        '0': note_int2char(key) + ' Major',
        '1': note_int2char(key) + ' minor',
    }.get(str(key // 12), 'undef')

# get chords of a key
def key_to_chords(key):
    return np.nonzero(np.transpose(chord_to_key)[np.argmax(key)])

def get_chords(midi_info):
    chords_predict = np.zeros(midi_info.shape[0], dtype=np.uint8)
    for bar_idx in range(midi_info.shape[0]):
        notes_stat = np.zeros(12, dtype=np.float)
        for channel_idx in range(MAX_NUM_OF_CHANNELS):
            if channel_idx != DRUM_CHANNEL:
                for info in midi_info[bar_idx, channel_idx]:
                    notes_stat[int(info[0] % 12)] += info[3]
        chords_score = np.dot(chords, notes_stat)
        chords_predict[bar_idx] = np.argmax(chords_score)
    #     with np.printoptions(precision=3, suppress=True):
    #         print(["%s: %f"%(note_int2char(idx), note) for idx, note in enumerate(notes_stat) if note > 0])
    #         print(name_chord(chord_predict))
    #         print([name_chord(chord) for chord in np.argsort(chords_score)[-10:]])
    return chords_predict

def get_key(chords_predict):
    chords_stat = np.zeros(chords.shape[0], dtype=np.int)
    for chord_predict in chords_predict:
        chords_stat[chord_predict] += 1
    key_stat = np.dot(chords_stat, chord_to_key)
    key_predict = np.argmax(key_stat)
    key_acc = np.sum(chords_stat[key_to_chords(key_stat)]) / np.sum(chords_stat)
    return key_predict, key_acc, chords_stat

def get_msg_non_note(midi_file):
    msg_non_note = np.empty((0,2))
    total_tick = 0
    ticks_per_beat = midi_file.ticks_per_beat
    current_tempo = 0

    for msg in midi_file:
        if msg.time != 0:
            total_tick += mido.second2tick(msg.time, ticks_per_beat, current_tempo)

        if msg.is_meta:
            # update tempo for 'second2tick'
            if msg.type == 'set_tempo':
                current_tempo = msg.tempo
            msg_non_note = np.vstack((msg_non_note, np.array([total_tick, msg])))
        else:
            if not is_note(msg):
                msg_non_note = np.vstack((msg_non_note, np.array([total_tick, msg])))
    return msg_non_note

def info2midi(midi_info, msg_non_note, ticks_per_beat):

    UINT32_MAX = 4294967296 - 1

    channels_block = np.empty((MAX_NUM_OF_CHANNELS), dtype=np.object)
    for i in range(MAX_NUM_OF_CHANNELS):
        channels_block[i] = np.empty((0, 5), dtype=np.uint32)

    ticks_per_beat = midi_file.ticks_per_beat

    for channel_idx in range(MAX_NUM_OF_CHANNELS):
        for bar_idx in range(midi_info.shape[0]):
            bar_tick = ticks_per_beat * bars_beat[bar_idx]
            for info in midi_info[bar_idx, channel_idx]:
                if info[6] == 0:
                    channels_block[channel_idx] = np.vstack((channels_block[channel_idx], [(bar_idx + info[2]) * bar_tick, 0, channel_idx, info[0], info[1]]))
                    channels_block[channel_idx] = np.vstack((channels_block[channel_idx], [(bar_idx + info[2] + info[3]) * bar_tick, 1, channel_idx, info[0], info[1]]))
                else:
                    if info[2] == 0:
                        channels_block[channel_idx] = np.vstack((channels_block[channel_idx], [(bar_idx + info[2] + info[3]) * bar_tick, 1, channel_idx, info[0], info[1]]))
                    else:
                        channels_block[channel_idx] = np.vstack((channels_block[channel_idx], [(bar_idx + info[2]) * bar_tick, 0, channel_idx, info[0], info[1]]))
            channels_block[channel_idx] = channels_block[channel_idx][channels_block[channel_idx][:,0].argsort()]

    first_msg = np.zeros((17, 2), dtype=np.uint32)
    # initialize the comparison array
    for channel_idx in range(MAX_NUM_OF_CHANNELS):
        if channels_block[channel_idx].shape[0] != 0:
            first_msg[channel_idx, 1] = channels_block[channel_idx][0, 0]
        else:
            first_msg[channel_idx, 1] = UINT32_MAX
    if msg_non_note.shape[0] != 0:
        first_msg[16, 1] = msg_non_note[0, 0]
    else:
        first_msg[16, 1] = UINT32_MAX


    midi_file_new = mido.MidiFile(type=0)
    midi_file_new.ticks_per_beat = midi_file.ticks_per_beat
    track_new = mido.MidiTrack()
    midi_file_new.tracks.append(track_new)

    total_ticks = 0

    while True:
        channel_to_append = np.argmin(first_msg[:, 1])
        if first_msg[channel_to_append, 1] == UINT32_MAX:
            break
        if channel_to_append < 16:
            info = channels_block[channel_to_append][first_msg[channel_to_append, 0]]

            tick = (int)(info[0])
            msg = mido.Message('note_on', channel=(int)(info[2]), note=(int)(info[3]), velocity=(int)(info[4]), time=0)
            if info[1] == 1:
                msg = mido.Message('note_off', channel=(int)(info[2]), note=(int)(info[3]), velocity=(int)(info[4]), time=0)

            if tick != total_ticks:
                msg.time = tick - total_ticks
                total_ticks = tick

            track_new.append(msg)

            first_msg[channel_to_append, 0] += 1
            if first_msg[channel_to_append, 0] < channels_block[channel_to_append].shape[0]:
                first_msg[channel_to_append, 1] = channels_block[channel_to_append][first_msg[channel_to_append, 0], 0]
            else:
                first_msg[channel_to_append, 1] = UINT32_MAX
        else:
            info = msg_non_note[first_msg[channel_to_append, 0]]

            tick = (int)(info[0])        
            msg = info[1]
            msg.time = 0

            if tick != total_ticks:
                msg.time = tick - total_ticks
                total_ticks = tick

            track_new.append(msg)

            first_msg[channel_to_append, 0] += 1
            if first_msg[channel_to_append, 0] < msg_non_note.shape[0]:
                first_msg[channel_to_append, 1] = msg_non_note[first_msg[channel_to_append, 0], 0]
            else:
                first_msg[channel_to_append, 1] = UINT32_MAX
    return midi_file_new
            

In [239]:
# input the file name
midi_string = 'sample1'
midi_file = mido.MidiFile('../sample_audio/%s.mid' % midi_string)

# get the beat and tempo in each bar
_, bars_tick, bars_beat, bars_tempo = get_bars_info(midi_file)

# separate notes to their relative bars
bars_note_info = get_bars_note_info(midi_file)

# generate the rhythm pattern[channel index, bar index]
# rhythm_pattern = get_rhythm_pattern(bars_note_info)

midi_info = get_midi_info(bars_note_info)

chords_predict = get_chords(midi_info)

key_predict, key_acc, chords_stat = get_key(chords_predict)

msg_non_note = get_msg_non_note(midi_file)

with np.printoptions(precision=3, suppress=True):
#     print(bars_tick)
#     print([bar_note_info[3] for bar_note_info in bars_note_info])
    print(midi_info[:,1][0:2])
    print(chords_predict[0:30])
    print([name_chord(chord_predict) for chord_predict in chords_predict[0:30]])
    print('\nKey: %s (accuracy: %f)' % (name_key(key_predict), key_acc))
    print('\nChord statistics')
    for chord_stat_idx, chord_stat in enumerate(chords_stat):
        if chord_stat != 0:
            print('%s\t: %d' % (name_chord(chord_stat_idx), chord_stat))
    print(msg_non_note)
    
    print(bars_beat)
    print(bars_tick)

[array([[38. , 88. ,  0.5,  0.5,  2. ,  2. ,  1. ]])
 array([[38.   , 88.   ,  0.   ,  0.002,  0.   ,  0.006,  1.   ],
       [38.   , 88.   ,  0.   ,  0.184,  0.   ,  0.738,  0.   ],
       [50.   , 88.   ,  0.188,  0.059,  0.75 ,  0.237,  0.   ],
       [38.   , 88.   ,  0.25 ,  0.184,  1.   ,  0.738,  0.   ],
       [50.   , 88.   ,  0.438,  0.059,  1.75 ,  0.237,  0.   ],
       [38.   , 88.   ,  0.5  ,  0.184,  2.   ,  0.738,  0.   ],
       [50.   , 88.   ,  0.688,  0.059,  2.75 ,  0.237,  0.   ],
       [38.   , 88.   ,  0.75 ,  0.184,  3.   ,  0.738,  0.   ],
       [50.   , 88.   ,  0.938,  0.059,  3.75 ,  0.237,  0.   ]])]
[ 2  2  2  2  2 14 38 43 29 21 33 43 57 38 38 43 81 33 33  7 53  2  2  2
  2  2  2  7 29  9]
['D1', 'D1', 'D1', 'D1', 'D1', 'D5', 'Dm', 'Gm', 'F', 'A5', 'A', 'Gm', 'A7', 'Dm', 'Dm', 'Gm', 'Am7', 'A', 'A', 'G1', 'F7', 'D1', 'D1', 'D1', 'D1', 'D1', 'D1', 'G1', 'F', 'A1']

Key: D minor (accuracy: 0.888199)

Chord statistics
D1	: 34
G1	: 5
A1	: 4
D5	: 1
G5	: 5


In [244]:
midi_file_new = info2midi(midi_info, msg_non_note, midi_file.ticks_per_beat)

import time    

# use 'print(mido.get_output_names())' to get your port's name and type it in the 'mido.open_output' bracket
port = None
port = mido.open_output('Microsoft GS Wavetable Synth 0')
play_audio = True

try:

    for msg in midi_file_new:
        
        # delay for playing
        if play_audio:
            time.sleep(msg.time)
            if not msg.is_meta:
                port.send(msg)
                print(msg)

    if port is not None:
        port.close()
    print('End of midi')
    
# action after pressing stop
except KeyboardInterrupt:
    
    if port is not None:
        port.close()
    print('Forced stop')

control_change channel=0 control=121 value=0 time=0
control_change channel=0 control=123 value=0 time=0
control_change channel=0 control=100 value=0 time=0
control_change channel=0 control=38 value=0 time=0
control_change channel=0 control=101 value=0 time=0
control_change channel=0 control=6 value=12 time=0
pitchwheel channel=0 pitch=0 time=0
program_change channel=0 program=81 time=0.11810999999999999
control_change channel=0 control=7 value=88 time=0
control_change channel=0 control=10 value=80 time=0
control_change channel=1 control=121 value=0 time=0
control_change channel=1 control=123 value=0 time=0
control_change channel=1 control=100 value=0 time=0
control_change channel=1 control=38 value=0 time=0
control_change channel=1 control=101 value=0 time=0
control_change channel=1 control=6 value=12 time=0
pitchwheel channel=1 pitch=0 time=0
program_change channel=1 program=30 time=0.11810999999999999
control_change channel=1 control=7 value=104 time=0
control_change channel=1 contro

note_off channel=0 note=26 velocity=95 time=0.30708599999999997
note_off channel=2 note=38 velocity=95 time=0
note_off channel=9 note=36 velocity=92 time=0
note_off channel=9 note=41 velocity=95 time=0
note_on channel=0 note=38 velocity=95 time=0.005905499999999999
note_on channel=2 note=50 velocity=95 time=0
note_on channel=9 note=36 velocity=92 time=0
note_on channel=9 note=41 velocity=95 time=0
note_off channel=1 note=38 velocity=88 time=0.033464499999999994
note_on channel=1 note=50 velocity=88 time=0.005905499999999999
note_off channel=0 note=38 velocity=95 time=0.11220449999999998
note_off channel=1 note=50 velocity=88 time=0
note_off channel=2 note=50 velocity=95 time=0
note_off channel=9 note=36 velocity=92 time=0
note_off channel=9 note=41 velocity=95 time=0
note_on channel=0 note=26 velocity=95 time=0.005905499999999999
note_on channel=1 note=38 velocity=88 time=0
note_on channel=2 note=38 velocity=95 time=0
note_on channel=9 note=38 velocity=105 time=0
note_off channel=0 not

note_off channel=0 note=26 velocity=95 time=0.30905449999999995
note_off channel=2 note=38 velocity=95 time=0
note_off channel=9 note=38 velocity=105 time=0
note_on channel=0 note=38 velocity=95 time=0.005905499999999999
note_on channel=2 note=50 velocity=95 time=0
note_on channel=9 note=45 velocity=79 time=0
note_off channel=1 note=38 velocity=88 time=0.033464499999999994
note_on channel=1 note=50 velocity=88 time=0.005905499999999999
note_off channel=0 note=38 velocity=95 time=0.11122024999999999
note_off channel=2 note=50 velocity=95 time=0
note_off channel=9 note=45 velocity=79 time=0
note_off channel=1 note=50 velocity=88 time=0.0009842499999999999
note_on channel=0 note=26 velocity=95 time=0.005905499999999999
note_on channel=1 note=38 velocity=88 time=0
note_on channel=2 note=38 velocity=95 time=0
note_on channel=9 note=36 velocity=92 time=0
note_on channel=9 note=41 velocity=95 time=0
note_off channel=0 note=26 velocity=95 time=0.30905449999999995
note_off channel=2 note=38 vel

note_off channel=0 note=26 velocity=95 time=0.30905449999999995
note_off channel=2 note=38 velocity=95 time=0
note_off channel=9 note=36 velocity=92 time=0
note_off channel=9 note=41 velocity=95 time=0
note_on channel=0 note=38 velocity=95 time=0.005905499999999999
note_on channel=2 note=50 velocity=95 time=0
note_on channel=9 note=41 velocity=95 time=0
note_on channel=9 note=36 velocity=92 time=0
note_off channel=1 note=38 velocity=88 time=0.033464499999999994
note_on channel=1 note=50 velocity=88 time=0.005905499999999999
note_off channel=0 note=38 velocity=95 time=0.11220449999999998
note_off channel=1 note=50 velocity=88 time=0
note_off channel=2 note=50 velocity=95 time=0
note_off channel=9 note=41 velocity=95 time=0
note_off channel=9 note=36 velocity=92 time=0
note_on channel=0 note=26 velocity=95 time=0.005905499999999999
note_on channel=1 note=38 velocity=88 time=0
note_on channel=2 note=38 velocity=95 time=0
note_on channel=9 note=38 velocity=105 time=0
note_on channel=9 note

note_off channel=0 note=31 velocity=95 time=0.30905449999999995
note_off channel=2 note=43 velocity=95 time=0
note_off channel=9 note=36 velocity=92 time=0
note_off channel=9 note=41 velocity=95 time=0
note_on channel=0 note=43 velocity=95 time=0.005905499999999999
note_on channel=2 note=55 velocity=95 time=0
note_on channel=9 note=36 velocity=92 time=0
note_on channel=9 note=41 velocity=95 time=0
note_off channel=1 note=43 velocity=88 time=0.033464499999999994
note_on channel=1 note=55 velocity=88 time=0.005905499999999999
note_off channel=0 note=43 velocity=95 time=0.11220449999999998
note_off channel=1 note=55 velocity=88 time=0
note_off channel=2 note=55 velocity=95 time=0
note_off channel=9 note=41 velocity=95 time=0
note_off channel=9 note=36 velocity=92 time=0
note_on channel=0 note=31 velocity=95 time=0.005905499999999999
note_on channel=1 note=43 velocity=88 time=0
note_on channel=2 note=43 velocity=95 time=0
note_on channel=9 note=38 velocity=105 time=0
note_on channel=9 note